#### Copyright 2019 The TensorFlow Hub Authors.

Licensed under the Apache License, Version 2.0 (the "License");

In [ ]:
# Copyright 2019 The TensorFlow Hub Authors. All Rights Reserved.
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.
# ==============================================================================

# Exploramos las incorporaciones con CORD-19 Swivel de TF-Hub


<table class="tfo-notebook-buttons" align="left">
  <td><a target="_blank" href="https://www.tensorflow.org/hub/tutorials/cord_19_embeddings_keras"><img src="https://www.tensorflow.org/images/tf_logo_32px.png">Ver en TensorFlow.org</a></td>
  <td><a target="_blank" href="https://colab.research.google.com/github/tensorflow/docs-l10n/blob/master/site/es-419/hub/tutorials/cord_19_embeddings_keras.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png">Ejecutar en Google Colab</a></td>
  <td>     <a target="_blank" href="https://github.com/tensorflow/docs-l10n/blob/master/site/es-419/hub/tutorials/cord_19_embeddings_keras.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png">Ver en GitHub</a>
</td>
  <td>     <a href="https://storage.googleapis.com/tensorflow_docs/docs-l10n/site/es-419/hub/tutorials/cord_19_embeddings_keras.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png">Descargar el bloc de notas</a>
</td>
  <td>     <a href="https://tfhub.dev/tensorflow/cord-19/swivel-128d/3"><img src="https://www.tensorflow.org/images/hub_logo_32px.png">Ver modelo de TF Hub</a>
</td>
</table>

El módulo para incorporaciones de texto CORD-19 Swivel de TF-Hub (https://tfhub.dev/tensorflow/cord-19/swivel-128d/3) fue creado para ayudar a los investigadores a analizar textos de lenguaje natural relacionado con la COVID-19. Estas incorporaciones fueron entrenadas con títulos, autores, resúmenes, contenidos y títulos de referencia de artículos [del conjunto de datos CORD-19](https://api.semanticscholar.org/CorpusID:216056360).

En este artículo haremos lo siguiente:

- Analizar semánticamente palabras similares en el espacio de incorporación.
- Entrenar un clasificador en el conjunto de datos SciCite con las incorporaciones CORD-19.


## Preparación


In [ ]:
import functools
import itertools
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import pandas as pd

import tensorflow as tf

import tensorflow_datasets as tfds
import tensorflow_hub as hub

from tqdm import trange

# Análisis de las incorporaciones

Comencemos por analizar las incorporaciones a través de cálculos y gráficos que muestren la matriz de correlación entre términos diferentes. Si las incorporaciones aprendieron a capturar correctamente el significado de palabras diferentes, los vectores de incorporación de palabras semánticamente similares deberían estar juntos. Echemos un vistazo a algunos de los términos relacionados con la COVID-19.

In [ ]:
# Use the inner product between two embedding vectors as the similarity measure
def plot_correlation(labels, features):
  corr = np.inner(features, features)
  corr /= np.max(corr)
  sns.heatmap(corr, xticklabels=labels, yticklabels=labels)

# Generate embeddings for some terms
queries = [
  # Related viruses
  'coronavirus', 'SARS', 'MERS',
  # Regions
  'Italy', 'Spain', 'Europe',
  # Symptoms
  'cough', 'fever', 'throat'
]

module = hub.load('https://tfhub.dev/tensorflow/cord-19/swivel-128d/3')
embeddings = module(queries)

plot_correlation(queries, embeddings)

Podemos ver que la incorporación capturó correctamente el significado de diferentes términos. Cada palabra es similar a otras de su mismo clúster (es decir, la correlación de "coronavirus" con "SARS" y "MERS" es alta), pero es diferente a los términos de otros clústeres (es decir, la similitud entre "SARS" y "Spain" (España) es cercana a 0).

Ahora veamos cómo se pueden usar estas incorporaciones para resolver tareas específicas.

## SciCite: clasificación de intención de citas

En esta sección mostramos cómo se pueden usar las incorporaciones para descargar tareas como las de clasificación de textos. Usaremos el [conjunto de datos SciCite](https://www.tensorflow.org/datasets/catalog/scicite) de los conjuntos de datos de TensorFlow para clasificar las intenciones de citas de publicaciones académicas. Dada una oración con una cita en una publicación académica, clasifique si el propósito de la cita es sobre antecedentes, métodos de uso o comparación de resultados.

In [ ]:
builder = tfds.builder(name='scicite')
builder.download_and_prepare()
train_data, validation_data, test_data = builder.as_dataset(
    split=('train', 'validation', 'test'),
    as_supervised=True)

In [ ]:
#@title Let's take a look at a few labeled examples from the training set
NUM_EXAMPLES =   10#@param {type:"integer"}

TEXT_FEATURE_NAME = builder.info.supervised_keys[0]
LABEL_NAME = builder.info.supervised_keys[1]

def label2str(numeric_label):
  m = builder.info.features[LABEL_NAME].names
  return m[numeric_label]

data = next(iter(train_data.batch(NUM_EXAMPLES)))


pd.DataFrame({
    TEXT_FEATURE_NAME: [ex.numpy().decode('utf8') for ex in data[0]],
    LABEL_NAME: [label2str(x) for x in data[1]]
})

## Entrenamiento de un clasificador de intenciones de citas

Entrenaremos un clasificador del [conjunto de datos SciCite](https://www.tensorflow.org/datasets/catalog/scicite) con Keras. Creemos un modelo que use las incorporaciones de CORD-19 con una capa de clasificación arriba.

In [ ]:
#@title Hyperparameters { run: "auto" }

EMBEDDING = 'https://tfhub.dev/tensorflow/cord-19/swivel-128d/3'  #@param {type: "string"}
TRAINABLE_MODULE = False  #@param {type: "boolean"}

hub_layer = hub.KerasLayer(EMBEDDING, input_shape=[], 
                           dtype=tf.string, trainable=TRAINABLE_MODULE)

model = tf.keras.Sequential()
model.add(hub_layer)
model.add(tf.keras.layers.Dense(3))
model.summary()
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

## Entrenamiento y evaluación del modelo

Entrenemos y evaluemos el modelo para ver el desempeño con la tarea de SciCite.

In [ ]:
EPOCHS = 35#@param {type: "integer"}
BATCH_SIZE = 32#@param {type: "integer"}

history = model.fit(train_data.shuffle(10000).batch(BATCH_SIZE),
                    epochs=EPOCHS,
                    validation_data=validation_data.batch(BATCH_SIZE),
                    verbose=1)

In [ ]:
from matplotlib import pyplot as plt
def display_training_curves(training, validation, title, subplot):
  if subplot%10==1: # set up the subplots on the first call
    plt.subplots(figsize=(10,10), facecolor='#F0F0F0')
    plt.tight_layout()
  ax = plt.subplot(subplot)
  ax.set_facecolor('#F8F8F8')
  ax.plot(training)
  ax.plot(validation)
  ax.set_title('model '+ title)
  ax.set_ylabel(title)
  ax.set_xlabel('epoch')
  ax.legend(['train', 'valid.'])

In [ ]:
display_training_curves(history.history['accuracy'], history.history['val_accuracy'], 'accuracy', 211)
display_training_curves(history.history['loss'], history.history['val_loss'], 'loss', 212)

## Evaluación del modelo

Veamos el rendimiento del modelo. Nos devolverá dos valores; la pérdida (un número que representa nuestro error, los valores bajos son mejores) y la exactitud.

In [ ]:
results = model.evaluate(test_data.batch(512), verbose=2)

for name, value in zip(model.metrics_names, results):
  print('%s: %.3f' % (name, value))

Podemos ver que la pérdida disminuye rápidamente, mientras que la exactitud aumenta también rápidamente. Grafiquemos algunos ejemplos para verificar cómo se relaciona la predicción con las etiquetas verdaderas:

In [ ]:
prediction_dataset = next(iter(test_data.batch(20)))

prediction_texts = [ex.numpy().decode('utf8') for ex in prediction_dataset[0]]
prediction_labels = [label2str(x) for x in prediction_dataset[1]]

predictions = [
    label2str(x) for x in np.argmax(model.predict(prediction_texts), axis=-1)]


pd.DataFrame({
    TEXT_FEATURE_NAME: prediction_texts,
    LABEL_NAME: prediction_labels,
    'prediction': predictions
})

Observamos que para esta muestra aleatoria, el modelo predice la etiqueta correcta la mayoría de las veces, lo cual indica que puede incorporar oraciones científicas bastante bien.

# ¿Qué sigue?

Ahora que logró entender un poco más sobre las incorporaciones con CORD-19 Swivel de TF-Hub, le recomendamos participar en la competencia CORD-19 Kaggle para contribuir con la adquisición de conocimientos científicos de textos académicos relacionados con la COVID-19.

- Participe en el [desafío CORD-19 Kaggle](https://www.kaggle.com/allen-institute-for-ai/CORD-19-research-challenge)
- Acceda a más información sobre [COVID-19 Open Research Dataset (CORD-19)](https://api.semanticscholar.org/CorpusID:216056360)
- Consulte la documentación y mucho más sobre las incorporaciones con TF-Hub en https://tfhub.dev/tensorflow/cord-19/swivel-128d/3
- Explore el espacio de incorporaciones CORD-19 con la [herramienta para proyectar incorporaciones de TensorFlow](http://projector.tensorflow.org/?config=https://storage.googleapis.com/tfhub-examples/tensorflow/cord-19/swivel-128d/3/tensorboard/projector_config.json)